In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
from scipy.stats import itemfreq
from sklearn.neighbors import KNeighborsClassifier
from google.colab import drive
from google.colab import files
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


##KNN Algorithm
###Characteristics
* Non-parametric ML algorithm. It means KNN doesn't have training phase to find optimal parameter.
* Therefore, training phase and predicing phase in 1 phase make the algotithm very show as the data larger and larger, also the dimension of vector is big.
* ...

###Idea
* Given a specific dataset, for each row in a N-D dimension vector and the label.
* Pre-processing dataset (normalization, standardization) into same scale (optional).
* For any new point in predicting phase, the algorithm finds the distance between that point and all other points in training set (L_1, L_2, L_inf).
* Base on K hyper-parameter, the algorithm will find K nearest neighbor and classify that point into which class.

In [0]:
class KNN:
  _metrics = {'euclidean' : '_l2_distance', 'manhattan': '_l1_distance', 'cosine': '_cosine_similarity'}
  def __init__(self, K, X, Y, metric='euclidean'):
    self.K = K
    assert type(X) is np.ndarray, "X must be a numpy array"
    assert type(Y) is np.ndarray, "Y must be a numpy array"
    self.X = X
    self.Y = Y
    self.classes = np.unique(Y)
    self.metric = metric

  def _l1_distance(self, X_new):
    # l1 = abs(x_1 - x_2) + abs(y_1 - y_2)
    return cdist(X_new, self.X, 'cityblock')

  def _l2_distance(self, X_new):
    # l2 = sqrt((x_1 - x_2)**2 + (y_1 - y_2)**2)
    return cdist(X_new, self.X, 'euclidean')

  def _cosine_similarity(self, X_new):
    # similarity = cos(alpha) = dot(A, B) / (len(A)*len(B))
    return cdist(X_new, self.X, 'cosine')

  def predict(self, X_new):
    assert type(X_new) is np.ndarray, "Use numpy array instead."
    assert X_new.shape[1] == self.X.shape[1], "Mismatch shape."
    if self.metric not in self._metrics.keys():
      self.metric = 'euclidean'
    func = getattr(self, self._metrics[self.metric])
    dist = func(X_new)
    dist = np.argsort(dist, axis=1)
    k_nearest = dist[:,:self.K]
    labels = self.Y[k_nearest]
    res = []
    for label in labels:
      label, count = np.unique(label, return_counts=True)
      res.append(label[np.argmax(count)])
    return np.array(res)
    

In [3]:
!ls sample_data/

anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [4]:
def experiment(X, Y, X_test, Y_test):
  print("--- Experiment ---")
  ks = [1, 3, 5, 7, 9, 11]
  metrics = ['manhattan', 'euclidean', 'cosine']
  for metric in metrics:
    for k in ks:
      knn = KNN(k, X, Y, metric=metric)
      Y_pred = knn.predict(X_test)
      
      print("KKN with K = %d and metric = %s | Accuracy: %f" % (k, metric, len(Y_test[Y_pred == Y_test]) / len(Y_test)))
  print("-"*50)

def main():
  path = "/content/gdrive/My Drive/Colab Notebooks/AIML/Tutorial/KNN/data/"
  df = pd.read_csv(path+"train.csv")
  X = df.loc[:, :].values
  Y = pd.read_csv(path+"trainDirection.csv").iloc[:,0].values
  
  print("X shape:", X.shape)
  print("Y shape:", Y.shape)
  
  df_test = pd.read_csv(path+"testing.csv")
  X_test = df_test.drop('Direction', axis=1).iloc[:, 1:].values
  Y_test = df_test.loc[:, 'Direction'].values
  
  print("X test shape:", X_test.shape)
  print("Y test shape:", Y_test.shape)
  
  debug = True
  
  if debug:
    experiment(X, Y, X_test, Y_test)
    return
  
  k = 3
  
  knn = KNN(k, X, Y, metric='manhattan')
  pred = knn.predict(X_test)
  
  print("Accuracy:", len(Y_test[pred == Y_test]) / len(Y_test))
  
  sk_knn = KNeighborsClassifier(n_neighbors=k, metric='manhattan')
  sk_knn.fit(X,Y)
  
  y_sk = sk_knn.predict(X_test)
  print("Sk-learn KNN accuracy:", len(Y_test[y_sk == Y_test]) / len(Y_test))
     
if __name__ == '__main__':
  main()

X shape: (998, 2)
Y shape: (998,)
X test shape: (252, 2)
Y test shape: (252,)
--- Experiment ---
KKN with K = 1 and metric = manhattan | Accuracy: 0.928571
KKN with K = 3 and metric = manhattan | Accuracy: 0.742063
KKN with K = 5 and metric = manhattan | Accuracy: 0.690476
KKN with K = 7 and metric = manhattan | Accuracy: 0.666667
KKN with K = 9 and metric = manhattan | Accuracy: 0.646825
KKN with K = 11 and metric = manhattan | Accuracy: 0.615079
KKN with K = 1 and metric = euclidean | Accuracy: 1.000000
KKN with K = 3 and metric = euclidean | Accuracy: 0.761905
KKN with K = 5 and metric = euclidean | Accuracy: 0.698413
KKN with K = 7 and metric = euclidean | Accuracy: 0.638889
KKN with K = 9 and metric = euclidean | Accuracy: 0.630952
KKN with K = 11 and metric = euclidean | Accuracy: 0.634921
KKN with K = 1 and metric = cosine | Accuracy: 0.476190
KKN with K = 3 and metric = cosine | Accuracy: 0.511905
KKN with K = 5 and metric = cosine | Accuracy: 0.547619
KKN with K = 7 and metric